In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-data-for-dummy-bank/loan_final313.csv
/kaggle/input/loan-condition-eda-data-cleansing/__results__.html
/kaggle/input/loan-condition-eda-data-cleansing/__notebook__.ipynb
/kaggle/input/loan-condition-eda-data-cleansing/df_for_use.pkl
/kaggle/input/loan-condition-eda-data-cleansing/custom.css
/kaggle/input/loan-condition-eda-data-cleansing/__output__.json
/kaggle/input/loan-condition-eda-data-cleansing/__results___files/__results___11_1.png


### Libraries


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler , Binarizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from time import time
import os, sys, gc, warnings, random, datetime
import math
warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold , KFold
# from ngboost import NGBRegressor
import lightgbm as lgb

In [2]:
df = pd.read_pickle("/kaggle/input/loan-condition-eda-data-cleansing/df_for_use.pkl")

### Utilities

In [3]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test, pred)
    auc = roc_auc_score(y_test, pred)
    print('Confusion Matrix')
    print(confusion)
    print('Auccuracy : {0:.4f}, Precision : {1:.4f} , Recall : {2:.4f} , F1_Score : {3:.4f}, ROC_AUC_Score : {4:.4f}'.format(accuracy , precision, recall, f1, auc))

In [4]:
thresholds = {0.3,0.35, 0.4, 0.45, 0.50, 0.55, 0.60}

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('threshold:', custom_threshold)
        get_clf_eval(y_test, custom_predict)

## get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

### train_test_split (Stratify)

In [7]:
X = df.drop('loan_condition_cat', axis=1)
y = df['loan_condition_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 2020, stratify = y)


In [8]:
y_trainin = y_train.to_frame()
train_df = pd.concat([X_train, y_trainin], axis=1)
y_testet = y_test.to_frame()
test_df = pd.concat([X_test, y_testet], axis=1)



In [9]:
X = train_df.drop('loan_condition_cat', axis=1)
y = train_df['loan_condition_cat']


### LightGBM with Startified 5 Fold

In [19]:
from time import time
from lightgbm import LGBMClassifier

In [22]:
params_lgb={'boosting_type':'gbdt',
           'objective': 'binary',
           'random_state':2020,
           'metric':'auc'}
k_fold=5
kf=StratifiedKFold(n_splits=k_fold,shuffle=True, random_state=2020)
training_start_time = time()
aucs=[]

for fold, (trn_idx,val_idx) in enumerate(kf.split(X,y)):
    start_time = time()
    print('Training on fold {}'.format(fold + 1))
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])
    clf = lgb.train(params_lgb, trn_data, num_boost_round=10000, valid_sets = [trn_data, val_data], 
                    verbose_eval=200, early_stopping_rounds=200)
    aucs.append(clf.best_score['valid_1']['auc'])
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
print('-' * 30)
print('Training is completed!.')
print("\n## Mean AUC_Score : ", np.mean(aucs))
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
# print(clf.best_params_)
print('-' * 30)

Training on fold 1
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.92345	valid_1's auc: 0.908191
[400]	training's auc: 0.938739	valid_1's auc: 0.913432
[600]	training's auc: 0.948142	valid_1's auc: 0.915654
[800]	training's auc: 0.955162	valid_1's auc: 0.916401
[1000]	training's auc: 0.9617	valid_1's auc: 0.917035
[1200]	training's auc: 0.966902	valid_1's auc: 0.917604
[1400]	training's auc: 0.971649	valid_1's auc: 0.917521
Early stopping, best iteration is:
[1228]	training's auc: 0.96753	valid_1's auc: 0.917685
Fold 1 finished in 0:01:55.836991
Training on fold 2
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.922997	valid_1's auc: 0.910163
[400]	training's auc: 0.938849	valid_1's auc: 0.91579
[600]	training's auc: 0.948085	valid_1's auc: 0.917315
[800]	training's auc: 0.955147	valid_1's auc: 0.918358
[1000]	training's auc: 0.960958	valid_1's auc: 0.918903
[1200]	training's auc: 0.966325	valid_1's auc: 0.91896

In [23]:
X_test = test_df.drop('loan_condition_cat', axis=1)
y_test = test_df['loan_condition_cat']

In [27]:
pred_rf = clf.predict(X_test)
auc = roc_auc_score(y_test,pred_rf)
print(' ROC_AUC_Score : {0:.4f}'.format (auc))

 ROC_AUC_Score : 0.9174
